In [1]:
# Hi @davidhao3300, thanks for submitting this issue and for the clear example. I was able to reproduce your result, which will make this much easier to dig into. Indeed the last line returns a Future for each Pandas DataFrame in the Dask DataFrame, and can be converted to concrete values with `.result()`:
# ```python
# ddf.compute(scheduler="threads")
# Out[2]: 
# 0    <Future: finished, type: pandas.core.frame.Dat...
# 1    <Future: finished, type: pandas.core.frame.Dat...
# dtype: object
# ddf.compute(scheduler="threads")[0].result()
# Out[3]: 
#    a
# 0  1
# ddf.compute(scheduler="threads")[1].result()
# Out[4]: 
#    a
# 1  2
# 2  3
# ```

In [2]:
from distributed import Client, LocalCluster
import pandas as pd
import dask.dataframe as dd

In [3]:
cluster = LocalCluster()

/Users/sarahj/mambaforge/envs/dask-mini-tutorial/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 61270 instead
  warnings.warn(


In [4]:
client=Client(cluster)

In [5]:
client.has_what()

Worker,Key count,Key list
tcp://127.0.0.1:61284,0,Expand
tcp://127.0.0.1:61285,0,Expand
tcp://127.0.0.1:61290,0,Expand
tcp://127.0.0.1:61293,0,Expand


In [6]:
client.close()

In [7]:
from distributed import Client, LocalCluster
import pandas as pd
import dask.dataframe as dd

cluster = LocalCluster()
client=Client(cluster)

df = pd.DataFrame({'a': [1,2,3]})

ddf = dd.from_pandas(df, npartitions=3)

ddf = client.persist(ddf)

# Works, and returns a DF as expected
print(ddf.compute())

# Returns a pandas Series with a row per-partition with a future as the result
print(ddf.compute(scheduler="threads"))

/Users/sarahj/mambaforge/envs/dask-mini-tutorial/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 61357 instead
  warnings.warn(


   a
0  1
1  2
2  3
0    <Future: finished, type: pandas.core.frame.Dat...
1    <Future: finished, type: pandas.core.frame.Dat...
dtype: object
